To backup database:  
`mongodump -d=ThreatenedFlora -o=20200713 --gzip`

In [1]:
import pandas as pd
import re
from pymongo import MongoClient
from bson import ObjectId

In [2]:
def mystring(item , depth = 0):
    out = ''
    mydepth = depth
    
    if type(item) == list:
        for it in item:
            out += '\n'
            newdepth = mydepth + 1
            out += mystring(it, newdepth)
    
    elif type(item) == dict:
        for key in item:
            out += '  ' * mydepth + key + ': '
            newdepth = mydepth + 1
            out += mystring(item[key], newdepth)

    else:
        out += '{0}'.format(item) + '\n'
    
    return out

In [3]:
client = MongoClient()
tflora = client.ThreatenedFlora
taxo = tflora.Taxonomy
samples = tflora.Samples

In [50]:
samples.find_one()

{'_id': ObjectId('5eff66707b5e9c69d6a8e1bf'),
 'myindex': 0,
 'source': 'Bosque_seco',
 'occurrenceID': None,
 'basisOfRecord': 'Espécimen preservado',
 'institutionCode': 'UCALDAS',
 'collectionCode': 'FAUC',
 'catalogNumber': 'FAUC6108',
 'institutionID': None,
 'occurrenceRemarks': 'Arbusto de 3-4 m. Flores azules, fruto con semillas rojas. Cultivado como ornamental en el CIAT',
 'recordNumber': None,
 'recordedBy': 'M. De Fraume',
 'reproductiveCondition': 'Floración y Fructificación',
 'occurrenceStatus': None,
 'disposition': None,
 'otherCatalogNumbers': None,
 'eventDate': '1990-06-06',
 'eventRemarks': None,
 'habitat': None,
 'continent': 'SA',
 'country': 'Colombia',
 'countryCode': 'CO',
 'stateProvince': 'Valle del Cauca',
 'county': 'Palmira',
 'municipality': 'Palmira',
 'locality': 'CIAT',
 'verbatimLocality': None,
 'minimumElevationInMeters': 980.0,
 'maximumElevationInMeters': 980.0,
 'verbatimLatitude': None,
 'verbatimLongitude': None,
 'verbatimCoordinateSystem': 

In [18]:
qu = taxo.aggregate([{'$group': {'_id': '$genus'}}]) #, {'query' : {'family': 'Asteraceae'}})
for q in qu:
    print(q)

{'_id': 'Hypericum'}
{'_id': 'Cnidoscolus'}
{'_id': 'Allophoron'}
{'_id': 'Comparettia'}
{'_id': 'Chaetolepis'}
{'_id': 'Pleuridium'}
{'_id': 'Macleania'}
{'_id': 'Cryptothecia'}
{'_id': 'Melocactus'}
{'_id': 'Cavanillesia'}
{'_id': 'Mandevilla'}
{'_id': 'Micropterygium'}
{'_id': 'Nowellia'}
{'_id': 'Obtegomeria'}
{'_id': 'Schizymenium'}
{'_id': 'Hydrocotyle'}
{'_id': 'Riccia'}
{'_id': 'Coespeletia'}
{'_id': 'Masdevallia'}
{'_id': 'Lupinus'}
{'_id': 'Thalloloma'}
{'_id': 'Luteolejeunea'}
{'_id': 'Sievekingia'}
{'_id': 'Libanothamnus'}
{'_id': 'Plutarchia'}
{'_id': 'Corella'}
{'_id': 'Dibrachiella'}
{'_id': 'Microdus'}
{'_id': 'Pentacalia'}
{'_id': 'Aspidosperma'}
{'_id': 'Niphogeton'}
{'_id': 'Lichenomphalia'}
{'_id': 'Caloplaca'}
{'_id': 'Colura'}
{'_id': 'Macrocarpaea'}
{'_id': 'Bejaria'}
{'_id': 'Barbula'}
{'_id': 'Draba'}
{'_id': 'Icmadophila'}
{'_id': 'Agonimia'}
{'_id': 'Mezobromelia'}
{'_id': 'Miconia'}
{'_id': 'Phlegmariurus'}
{'_id': 'Plagiochila'}
{'_id': 'Inga'}
{'_id': 'Phy

In [56]:
qu = samples.aggregate([
    {'$match':
        {'$and': [
            {'source': 'Frailejones_g'},
            {'activated': 0}
        ]}
    },
    {"$lookup":
        {"localField": "taxonID",
        "from" : "Taxonomy",
        "foreignField": "_id",
        "as": "taxon"}
    },
    {'$match':
        {'$or': [
            {'taxon.genus': 'Espeletia'},
            {'taxon.genus': 'Espeletiopsis'},
            {'taxon.genus': 'Coespeletia'},
            {'taxon.genus': 'Paramiflos'},
            {'taxon.genus': 'Ruilopezia'},
            {'taxon.genus': 'Libanothamnus'},
            {'taxon.genus': 'Tamania'}
        ]},
    },
    {'$project':{
        'source': 1,
        'recordedBy': 1,
        'recordNumber': 1,
        'activated': 1,
        'comment' : 1,
        'taxon.genus': 1
        }
    },
    #{'$group': {'_id': '$taxon.genus'}}
    ])

for x in qu:
    print(mystring(x))
    """
    samples.update_one({"_id": x['_id']},
        {"$set": {"comment": "Georreferenciación dudosa."}}
        )
    """


_id: 5eff66707b5e9c69d6a91ef9
source: Frailejones_g
recordNumber: 425
recordedBy: J. Rauscher; A. Repizzo; P. ánchez
activated: 0
comment: Georreferenciación dudosa.
taxon: 
    genus: Espeletia

_id: 5eff66707b5e9c69d6a91f32
source: Frailejones_g
recordNumber: 9483A
recordedBy: Cuatrecasas, José
activated: 0
comment: Georreferenciación dudosa.
taxon: 
    genus: Espeletia

_id: 5eff66707b5e9c69d6a91fe3
source: Frailejones_g
recordNumber: 10364A
recordedBy: Cuatrecasas, José
activated: 0
comment: Georreferenciación dudosa.
taxon: 
    genus: Espeletia

_id: 5eff66707b5e9c69d6a92072
source: Frailejones_g
recordNumber: 12653A
recordedBy: Cuatrecasas, José
activated: 0
comment: Georreferenciación dudosa.
taxon: 
    genus: Espeletia

_id: 5eff66707b5e9c69d6a92073
source: Frailejones_g
recordNumber: 12653C
recordedBy: Cuatrecasas, José
activated: 0
comment: Georreferenciación dudosa.
taxon: 
    genus: Espeletia

_id: 5eff66707b5e9c69d6a92172
source: Frailejones_g
recordNumber: 406
recorde

In [6]:
colls = pd.DataFrame.from_dict(dat)

In [12]:
colls = colls.drop(index=colls[colls.recordedBy.isna()].index)

In [22]:
def getColl(row):
    if row.source == 'Paramos':
        bits = re.split(r'\|', row.recordedBy)
    else:
        bits = re.split(r',', row.recordedBy)
    return bits[0]

In [26]:
colls.head(30).apply(getColl, axis =1)

0              David Sanín
1           M. Montoya J. 
2        William G. Vargas
3          G. Arbelaez S. 
4                C. Vélez 
5       Juan Manuel Duque 
6              A. de Wilde
7     L. M. Álvarez Mejía 
8            Sandra Arango
9       Heliodoro Sánchez 
10      Heliodoro Sánchez 
11           César Barbosa
12         Julio Betancur 
13            F. Gonzáles 
14          Hermano Daniel
15        José Cuatrecasas
16        José Cuatrecasas
17          M. L. Becking 
18           M. L. Becking
19          M. L. Becking 
20           D. Macias P. 
21       José Cuatrecasas 
22        José Cuatrecasas
23      Harriet G. Barclay
24     Gustavo Huertas G. 
25        Santiago Díaz P.
26     Gustavo Huertas G. 
27              M. Velayos
28              D. Macías 
29         E. L. Muñoz E. 
dtype: object

In [19]:
colls.head(30)

,source,recordedBy,recordNumber
0,Paramos,David Sanín,814
1,Paramos,M. Montoya J. | S. Ramírez E. | E. Gonzáles | ...,631
2,Paramos,William G. Vargas,WV 1287
3,Paramos,G. Arbelaez S. | C. Vélez N | N. Carvajal D | ...,GA 630
4,Paramos,C. Vélez | M. L. Chacón,4556
5,Paramos,Juan Manuel Duque | D. F. Londoño,834
6,Paramos,A. de Wilde,5470
7,Paramos,L. M. Álvarez Mejía | A. F. Bohórquez O.,LMAM 3965
8,Paramos,Sandra Arango,SA 281
9,Paramos,Heliodoro Sánchez | Jorge Hernández,453


In [78]:
qu = taxo.aggregate([
    {"$match": { "$and":[
        {"genus": "Magnolia"},
        {"specificEpithet": "hernandezii"}
        ]}
    },
    {"$lookup":
        {"localField": "_id",
        "from" : "Samples",
        "foreignField": "taxonID",
        "as": "record"}
    },
    {"$project": {
        "_id": 1,
        "family": 1,
        "genus": 1,
        "specificEpithet": 1,
        "TaxonID": 1,
        "record._id": 1,
        "record.recordedBy": 1,
        "record.recordNumber": 1,
        "record.decimalLatitude": 1,
        "record.decimalLongitude": 1,
        "record.country" : 1,
        "record.stateProvince": 1,
        "record.activated": 1
        }
    }
])

for row in qu:
    print(mystring(row))

_id: 5eff663d7b5e9c69d6a8df61
genus: Magnolia
specificEpithet: hernandezii
family: Magnoliaceae
record: 
    _id: 5eff66707b5e9c69d6a8fbb3
    recordNumber: 4419
    recordedBy: Vargas W.
    country: Colombia
    stateProvince: Risaralda
    decimalLatitude: None
    decimalLongitude: None
    activated: 1

    _id: 5eff66707b5e9c69d6a8fbb4
    recordNumber: 4014
    recordedBy: Vargas W.
    country: Colombia
    stateProvince: Risaralda
    decimalLatitude: None
    decimalLongitude: None
    activated: 1

    _id: 5eff66707b5e9c69d6a8fbbd
    recordNumber: 48
    recordedBy: Gómez C.
    country: Colombia
    stateProvince: Antioquia
    decimalLatitude: 6.3
    decimalLongitude: -75.7333333
    activated: 1

    _id: 5eff66707b5e9c69d6a8fbfe
    recordNumber: 28
    recordedBy: Mariño Y.
    country: Colombia
    stateProvince: Quindío
    decimalLatitude: 4.67759
    decimalLongitude: -75.621417
    activated: 1

    _id: 5eff66707b5e9c69d6a8fc00
    recordNumber: 857
    recorde

In [81]:
samples.update({"_id": ObjectId('5eff66707b5e9c69d6a8fd27')},
    {"$set": {"comment": "Georreferenciado por fuera del pais.", "activated": 0}}
)

/home/nelson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


{'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}

In [80]:
for i in samples.find({"_id": ObjectId('5eff66707b5e9c69d6a8fd27')}):
    print(mystring(i))

_id: 5eff66707b5e9c69d6a8fd27
myindex: 7016
source: Magnolias
occurrenceID: None
basisOfRecord: Espécimen Preservado
institutionCode: JBOTANICOMED
collectionCode: JAUM
catalogNumber: 38834
institutionID: 890980756-2
occurrenceRemarks: Árbol de 30 m de alto, tronco recto. DAP 33 cm, hojas verde oscuro, frutos verde claro
recordNumber: 10227
recordedBy: Devia W.
reproductiveCondition: None
occurrenceStatus: None
disposition: Duplicado en otra colección biológica 
otherCatalogNumbers: TULV 16539
eventDate: 2004-07-01/2004-07-31
eventRemarks: None
habitat: None
continent: SA
country: Colombia
countryCode: CO
stateProvince: Valle del Cauca
county: Calima
municipality: Darién
locality: Vereda San José, finca El Carmen a 4 km perímetro urbano
verbatimLocality: None
minimumElevationInMeters: 2000.0
maximumElevationInMeters: 2000.0
verbatimLatitude: 3°57'31"N
verbatimLongitude: 78°53'34"W
verbatimCoordinateSystem: grados minutos segundos
decimalLatitude: 3.9589826
decimalLongitude: -78.4896024


In [50]:
qu = samples.aggregate([
    {"$match": { "$and": [
        {"decimalLatitude": {"$gt": 13.3, "$lt": 13.4}},
        {"decimalLongitude": {"$gt": -81.3, "$lt": -81.2}}
        ]},
    },
    {"$lookup":
        {"localField": "taxonID",
        "from" : "Taxonomy",
        "foreignField": "_id",
        "as": "name"}
    },
    {"$project": {
        "name._id": 1,
        "name.family": 1,
        "name.genus": 1,
        "name.specificEpithet": 1,
        "TaxonID": 1,
        "recordedBy": 1,
        "recordNumber": 1,
        "decimalLatitude": 1,
        "decimalLongitude": 1,
        "country" : 1,
        "stateProvince": 1,
        "activated": 1
        }
    },
    {"$match": {"name.family": "Arecaceae"}},
    {"$limit": 5}
])

for spp in qu:
    print(mystring(spp))

_id: 5eff66707b5e9c69d6a8f443
recordNumber: 79718
recordedBy: Gentry Alwyn; R. Ortiz
country: Colombia
stateProvince: Archipiélago de San Andrés, Providencia y Santa Catalina
decimalLatitude: 13.3561556
decimalLongitude: -81.221566
activated: 0
name: 
    _id: 5eff663d7b5e9c69d6a8ddb5
    genus: Coccothrinax
    specificEpithet: argentata
    family: Arecaceae



In [39]:
qu = samples.aggregate([
    {"$match": 
        {"taxonID": ObjectId('5eff663d7b5e9c69d6a8e0ad')},
    },
    {"$project": {
        "TaxonID": 1,
        "recordedBy": 1,
        "recordNumber": 1,
        "decimalLatitude": 1,
        "decimalLongitude": 1,
        "country" : 1,
        "stateProvince": 1,
        "activated": 1
        }
    }
])

for spp in qu:
    print(mystring(spp))

_id: 5eff66707b5e9c69d6a8f451
recordNumber: 1159
recordedBy: Barbosa, C.
country: Colombia
stateProvince: Chocó
decimalLatitude: 7.813
decimalLongitude: -77.201
activated: 1

_id: 5eff66707b5e9c69d6a8f60f
recordNumber: 18707
recordedBy: Schultes, R. E.; Cabrera, I.
country: Colombia
stateProvince: Antioquia
decimalLatitude: 7.4108
decimalLongitude: -76.5762
activated: 1

_id: 5eff66707b5e9c69d6a8f610
recordNumber: 9995
recordedBy: Duke James A.
country: Colombia
stateProvince: Antioquia
decimalLatitude: 8.1
decimalLongitude: -76.73
activated: 1

_id: 5eff66707b5e9c69d6a8f611
recordNumber: 582
recordedBy: Jiménez-E. Néstor David; Cristina Estupiñan; Pedro Hernández; Rosalba Ruíz
country: Colombia
stateProvince: Córdoba
decimalLatitude: 7.6658611
decimalLongitude: -76.1020556
activated: 1

_id: 5eff66707b5e9c69d6a8f612
recordNumber: 406
recordedBy: Estupiñán-G. Ana Cristina; David Jiménez-E.; Pedro Hernández; Rosalba Ruíz; Mario Molina
country: Colombia
stateProvince: Córdoba
decimalLati

## Export csv table for analysis

In [25]:
qu = samples.aggregate([
    {"$match": {"$and": [
        {"activated": 1},
        {"decimalLongitude": {"$exists": True, "$ne": None}},
        {"decimalLatitude": {"$exists": True, "$ne": None}}
        ]}
    },
    {"$lookup":
        {"localField": "taxonID",
        "from" : "Taxonomy",
        "foreignField": "_id",
        "as": "name"}
    },
    {"$project": 
        {"name._id": 1,
        "name.family": 1,
        "name.genus": 1,
        "name.specificEpithet": 1,
        "name.project": 1,
        "name.distribution": 1,
        "name.threatStatusScope": 1,
        "TaxonID": 1,
        "recordedBy": 1,
        "recordNumber": 1,
        "decimalLatitude": 1,
        "decimalLongitude": 1}
    },
    {"$match":
        {"$and": [
            {"name.distribution": 'Endemica'},
            {"name.project": "Bosque_seco"}
        ]}
    }#,
    #{"$limit": 5}
])

dat2pan = {'Taxon': [], 'Latitude' : [], 'Longitude': [], 'Collector': [], 'CollectionNumber': []}

for row in qu:
    dat2pan['Latitude'].append(row['decimalLatitude'])
    dat2pan['Longitude'].append(row['decimalLongitude'])
    dat2pan['Collector'].append(row['recordedBy'])
    dat2pan['CollectionNumber'].append(row['recordNumber'])
    species = row['name'][0]['genus'] + ' ' + row['name'][0]['specificEpithet']
    #print(species)
    dat2pan['Taxon'].append(species)
    '''
    for na in spp['name']:
        #print(species, sam['decimalLongitude'], sam['decimalLatitude'])
        species = na['genus'] + " " + na['specificEpithet']
        dat2pan['Taxon'].append(species)
    
    #print(mystring(spp))    
    '''
colls = pd.DataFrame.from_dict(dat2pan)
colls.to_csv('bosque_seco.csv', index=False)

In [20]:
colls.shape

(1861, 5)

In [93]:
qu = taxo.aggregate([
    {"$match": {"family": "Arecaceae"}},
    {"$project": 
        {"_id": 1,
        "genus": 1,
        "specificEpithet": 1,
        'threatStatus': 1,
        'threatCriteria': 1}
    }#,
    #{"$limit": 5}
])

dat2pan = {'Taxon': [], 'Category' : [], 'Criteria': []}

for spp in qu:
    species = spp['genus'] + " " + spp['specificEpithet']
    dat2pan['Taxon'].append(species)
    dat2pan['Category'].append(spp['threatStatus'])
    dat2pan['Criteria'].append(spp['threatCriteria'])

    #print(species, spp['threatCriteria'])
    #print(species, spp['threatStatus'], spp['threatCriteria'])
    #print(mystring(spp))
    
pd.DataFrame.from_dict(dat2pan).to_csv('Arecaceae_categories.csv', index=False)

----

In [38]:
for i in samples.find({
    'taxonID': ObjectId('5eff663d7b5e9c69d6a8e198')}):
    print(i['recordedBy'])

W. Devia; J. Adarve
M. Heredia
W. Vargas; R. Gutiérrez
W. Vargas
W. Vargas; E. Mendez
W. Devia; J. Adarve
A. Gentry
C. Hughes
G. Reina; O. Meneses; J. Perafán
P. Silverstone; N. Paz; M. Heredia
J. E. Ramos; L. H. Ramos
J. E. Ramos; L. H. Ramos
J. E. Ramos; L. H. Ramos
P. Hutchinson; J. Idrobo; J. Kenneth
W. Devia
A. Gentry; P. Keating; M. Heredia; M. Monsalve
C. Ruiz; O. Meneses; G. Reina
J. E. Ramos; C. Reyna
G. Reina
D. L. Hurtado; D. Múnar; C. L. Sandoval; L. Piedrahíta
W. Vargas
None


In [49]:
for i in samples.find({
    'recordedBy': {'$regex': 'Espinal'},
    'recordNumber': {'$regex': '639'}}):
    print(mystring(i))

_id: 5eff66707b5e9c69d6a8e536
myindex: 887
source: Bosque_seco
occurrenceID: None
basisOfRecord: Espécimen preservado
institutionCode: UNAL
collectionCode: COL
catalogNumber: COL98652
institutionID: None
occurrenceRemarks: None
recordNumber: 639
recordedBy: S. Espinal
reproductiveCondition: None
occurrenceStatus: None
disposition: None
otherCatalogNumbers: None
eventDate: None
eventRemarks: None
habitat: None
continent: SA
country: Colombia
countryCode: CO
stateProvince: Antioquia
county: Santa Fé de Antioquia
municipality: None
locality: None
verbatimLocality: None
minimumElevationInMeters: 650.0
maximumElevationInMeters: 750.0
verbatimLatitude: 6º 33' 47" N
verbatimLongitude: 75º 49' 12" W
verbatimCoordinateSystem: grados, minutos, segundos
decimalLatitude: 7.0
decimalLongitude: -75.82
geodeticDatum: WGS84
coordinateUncertaintyInMeters: 220
georeferencedBy: Universidad Nacional de Colombia
georeferencedDate: 2007-02-14
georeferenceProtocol: Método radio-punto; Dead Reckoning; CHAPMAN

In [47]:
samples.update_many({'_id': ObjectId('5eff66707b5e9c69d6a8e4f5')}, 
    {"$set": {"activated": 0, "comment": "Probablemente cultivada."}})

In [50]:
taxo.update_one({"_id": ObjectId('5eff663d7b5e9c69d6a8df6a')}, 
    {"$set": {"distribution": "Nativa"}}
)

In [58]:
taxo.update_many({"$and": [{'project': 'Frailejones'}, {'distribution': 'Nativa'}]}, 
    {"$set": {"threatStatusScope": "National"}}
)

In [46]:
for t in taxo.find({'_id': ObjectId('5eff663d7b5e9c69d6a8dd79')}):
    print(mystring(t))

_id: 5eff663d7b5e9c69d6a8dd79
genus: Bulnesia
specificEpithet: carrapo
project: Bosque_seco
kingdom: Plantae
phylum: Tracheophyta
family: Zygophyllaceae
scientificName: Bulnesia carrapo
infraspecificEpithet: None
taxonRank: Especie
scientificNameAuthorship: Killip & Dugand
threatStatus: LC
threatCriteria: None
threatStatusScope: Global
distribution: Endemica
comments: None



In [83]:
taxo.distinct('project')

[None,
 'Bosque_seco',
 'Brom-Ericad',
 'Frailejones',
 'Magnolias',
 'Palmas',
 'Paramo',
 'Vedas',
 'Zamias']

In [19]:
samples.find_one()

{'_id': ObjectId('5eff66707b5e9c69d6a8e1bf'),
 'myindex': 0,
 'source': 'Bosque_seco',
 'occurrenceID': None,
 'basisOfRecord': 'Espécimen preservado',
 'institutionCode': 'UCALDAS',
 'collectionCode': 'FAUC',
 'catalogNumber': 'FAUC6108',
 'institutionID': None,
 'occurrenceRemarks': 'Arbusto de 3-4 m. Flores azules, fruto con semillas rojas. Cultivado como ornamental en el CIAT',
 'recordNumber': None,
 'recordedBy': 'M. De Fraume',
 'reproductiveCondition': 'Floración y Fructificación',
 'occurrenceStatus': None,
 'disposition': None,
 'otherCatalogNumbers': None,
 'eventDate': '1990-06-06',
 'eventRemarks': None,
 'habitat': None,
 'continent': 'SA',
 'country': 'Colombia',
 'countryCode': 'CO',
 'stateProvince': 'Valle del Cauca',
 'county': 'Palmira',
 'municipality': 'Palmira',
 'locality': 'CIAT',
 'verbatimLocality': None,
 'minimumElevationInMeters': 980.0,
 'maximumElevationInMeters': 980.0,
 'verbatimLatitude': None,
 'verbatimLongitude': None,
 'verbatimCoordinateSystem': 

In [102]:
client.close()